# Python による日本語自然言語処理
http://www.nltk.org/book-jp/ch12.html

# Mecab
Japanese lanaguage text segmentation library

Official Documentation: https://taku910.github.io/mecab/

## Install packages

- Install Mecab
- Install Ipadic
- Install mecab python

In [3]:
# Install Mecab

# !brew install mecab

In [4]:
# Ipadic is a dictionary used in the heart of mecab
# !brew install mecab-ipadic

Updating Homebrew...
To reinstall 2.7.0-20070801, run `brew reinstall mecab-ipadic`


In [3]:
# Install a mecab library in Python

# !pip install --upgrade pip
# !pip install mecab-python3
# !pip install urllib3
# !pip install beautifulsoup4

In [4]:
# Install NEologd - Neologism dictionary for MeCab
# doc: https://github.com/neologd/mecab-ipadic-neologd

# !git clone https://github.com/neologd/mecab-ipadic-neologd.git
# !cd ~/mecab-ipadic-neologd&&ls&&echo 'your-password' | sudo -S bin/install-mecab-ipadic-neologd -y

## Useful configuration commands

In [5]:
#Check the location of mecab
!mecab-config --sysconfdir 
# mecabrc is a config file
!ls $(mecab-config --sysconfdir)

/usr/local/etc
bash_completion.d mecabrc           nginx             wgetrc
colordiffrc       mecabrc-e         openssl
gitconfig         mecabrc~          openssl@1.1


In [6]:
#Check the location of dictionary
!mecab-config --dicdir 
#Look up what dictionary are installed
!ls $(mecab-config --dicdir) 
# Current dictionary
!mecab -D

# !cat $(mecab-config --sysconfdir)'/mecabrc'
# Change the dictionary from ipadic to neologd in the config file
# !sed -i -e 's/\/ipadic/\/mecab-ipadic-neologd/g' $(mecab-config --sysconfdir)'/mecabrc'
# Change the dictionary back to ipadic in the config file
# !sed -i -e 's/\/mecab-ipadic-neologd/\/ipadic/g' $(mecab-config --sysconfdir)'/mecabrc'

/usr/local/lib/mecab/dic
ipadic               mecab-ipadic-neologd userdic
filename:	/usr/local/lib/mecab/dic/ipadic/sys.dic
version:	102
charset:	utf8
type:	0
size:	392126
left size:	1316
right size:	1316

filename:	/usr/local/lib/mecab/dic/userdic/add.dic
version:	102
charset:	utf-8
type:	1
size:	1
left size:	1316
right size:	1316



In [6]:
# Import library
import MeCab
import urllib3
from bs4 import BeautifulSoup

## Simple Examples 

- Python
- Command line
- Analysis Options: wakati and chasen

In [9]:
# Very first naive example
mecab = MeCab.Tagger() 
mecab.parse("すもももももももものうち").split()

['すもも',
 '名詞,一般,*,*,*,*,すもも,スモモ,スモモ',
 'も',
 '助詞,係助詞,*,*,*,*,も,モ,モ',
 'もも',
 '名詞,一般,*,*,*,*,もも,モモ,モモ',
 'も',
 '助詞,係助詞,*,*,*,*,も,モ,モ',
 'もも',
 '名詞,一般,*,*,*,*,もも,モモ,モモ',
 'の',
 '助詞,連体化,*,*,*,*,の,ノ,ノ',
 'うち',
 '名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ',
 'EOS']

In [12]:
# -O wakati just seperate the text into words
wakati = MeCab.Tagger("-O wakati") 
wakati.parse("すもももももももものうち").split()

['すもも', 'も', 'もも', 'も', 'もも', 'の', 'うち']

In [15]:
# -O chasen provides details
chasen = MeCab.Tagger("-O chasen") 
chasen.parse("すもももももももものうち").split()

['すもも',
 'スモモ',
 'すもも',
 '名詞-一般',
 'も',
 'モ',
 'も',
 '助詞-係助詞',
 'もも',
 'モモ',
 'もも',
 '名詞-一般',
 'も',
 'モ',
 'も',
 '助詞-係助詞',
 'もも',
 'モモ',
 'もも',
 '名詞-一般',
 'の',
 'ノ',
 'の',
 '助詞-連体化',
 'うち',
 'ウチ',
 'うち',
 '名詞-非自立-副詞可能',
 'EOS']

In [16]:
%%bash
# Naive coding
mecab
すもももももももものうち

すもも	名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
うち	名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
EOS


In [12]:
%%bash
# Just seperation
mecab -O wakati
すもももももももものうち

すもも も もも も もも の うち 


In [13]:
%%bash
# Details
mecab -O chasen
すもももももももものうち

すもも	スモモ	すもも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
の	ノ	の	助詞-連体化		
うち	ウチ	うち	名詞-非自立-副詞可能		
EOS


In [17]:
%%bash
# Pass a variable to stdin
sample="生麦生米生卵"
mecab <<< $sample 

生麦	名詞,固有名詞,人名,姓,*,*,生麦,ナマムギ,ナマムギ
生	接頭詞,名詞接続,*,*,*,*,生,ナマ,ナマ
米	名詞,固有名詞,地域,国,*,*,米,ベイ,ベイ
生	名詞,接尾,一般,*,*,*,生,セイ,セイ
卵	名詞,一般,*,*,*,*,卵,タマゴ,タマゴ
EOS


In [18]:
%%bash
# # Using << is known as here-document structure
# cat > input.txt <<EOF
# 今日は良い天気ですね。
# $(date +%T)にめっちゃ走ったのに、電車に間に合わなかった。
# EOF

# Take a file as input and generate another file as output
mecab -O wakati -o output.txt input.txt
cat output.txt

今日 は 良い 天気 です ね 。 
16 : 42 : 28 に めっちゃ 走っ た のに 、 電車 に 間に合わ なかっ た 。 


## mecab-ipadic-NEologd

- \- d option is the key to switch the dictionary just one time

In [16]:
%%bash
#Just ipadic
# mecab -d /usr/local/lib/mecab/dic/ipadic <<<$sample
# mecab-ipadic-NEologd
sample="生麦生米生卵"
mecab -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd <<<$sample

生麦生米生卵	名詞,固有名詞,一般,*,*,*,生麦生米生卵,ナマムギナマゴメナマタマゴ,ナマムギナマゴメナマタマゴ
EOS


In [17]:
# Python with just seperation
sample="生麦生米生卵"
wakati = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -O wakati')
wakati.parse(sample).split()

['生麦生米生卵']

## mecab-ipadic vs mecab-ipadic-NEologd

In [18]:
# mecab-ipadic only
import MeCab
mecab = MeCab.Tagger()
mecab.parse("国会議員Youtuberの立花孝志がNHKとKaggleをぶっ壊す").split()

['国会',
 '名詞,一般,*,*,*,*,国会,コッカイ,コッカイ',
 '議員',
 '名詞,一般,*,*,*,*,議員,ギイン,ギイン',
 'Youtuber',
 '名詞,一般,*,*,*,*,*',
 'の',
 '助詞,連体化,*,*,*,*,の,ノ,ノ',
 '立花',
 '名詞,固有名詞,人名,姓,*,*,立花,タチバナ,タチバナ',
 '孝志',
 '名詞,固有名詞,人名,名,*,*,孝志,タカシ,タカシ',
 'が',
 '助詞,格助詞,一般,*,*,*,が,ガ,ガ',
 'NHK',
 '名詞,一般,*,*,*,*,*',
 'と',
 '助詞,並立助詞,*,*,*,*,と,ト,ト',
 'Kaggle',
 '名詞,一般,*,*,*,*,*',
 'を',
 '助詞,格助詞,一般,*,*,*,を,ヲ,ヲ',
 'ぶっ',
 '接頭詞,動詞接続,*,*,*,*,ぶっ,ブッ,ブッ',
 '壊す',
 '動詞,自立,*,*,五段・サ行,基本形,壊す,コワス,コワス',
 'EOS']

In [20]:
# mecab-ipadic-NEologd
neologd = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
neologd.parse("国会議員Youtuberの立花孝志がNHKとKaggleをぶっ壊す").split()

['国会議員',
 '名詞,固有名詞,一般,*,*,*,国会議員,コッカイギイン,コッカイギーン',
 'Youtuber',
 '名詞,固有名詞,一般,*,*,*,Youtuber,ユーチューバー,ユーチューバー',
 'の',
 '助詞,連体化,*,*,*,*,の,ノ,ノ',
 '立花孝志',
 '名詞,固有名詞,人名,一般,*,*,立花孝志,タチバナタカシ,タチバナタカシ',
 'が',
 '助詞,格助詞,一般,*,*,*,が,ガ,ガ',
 'NHK',
 '名詞,固有名詞,一般,*,*,*,NHK,エヌエイチケイ,エヌエイチケイ',
 'と',
 '助詞,並立助詞,*,*,*,*,と,ト,ト',
 'Kaggle',
 '名詞,一般,*,*,*,*,*',
 'を',
 '助詞,格助詞,一般,*,*,*,を,ヲ,ヲ',
 'ぶっ',
 '接頭詞,動詞接続,*,*,*,*,ぶっ,ブッ,ブッ',
 '壊す',
 '動詞,自立,*,*,五段・サ行,基本形,壊す,コワス,コワス',
 'EOS']

In [20]:
neologd = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

text = '国会議員Youtuberの立花孝志がNHKとKaggleをぶっ壊す'
node = neologd.parseToNode(text)

while node:
    word=node.surface 
    feature =node.feature.split(',') # Use split to put each element in a list
    print(word)
    print(','.join(feature)) # Use join to put together every element of the list
    node = node.next


BOS/EOS,*,*,*,*,*,*,*,*
国会議員
名詞,固有名詞,一般,*,*,*,国会議員,コッカイギイン,コッカイギーン
Youtuber
名詞,固有名詞,一般,*,*,*,Youtuber,ユーチューバー,ユーチューバー
の
助詞,連体化,*,*,*,*,の,ノ,ノ
立花孝志
名詞,固有名詞,人名,一般,*,*,立花孝志,タチバナタカシ,タチバナタカシ
が
助詞,格助詞,一般,*,*,*,が,ガ,ガ
NHK
名詞,固有名詞,一般,*,*,*,NHK,エヌエイチケイ,エヌエイチケイ
と
助詞,並立助詞,*,*,*,*,と,ト,ト
Kaggle
名詞,一般,*,*,*,*,*
を
助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
ぶっ
接頭詞,動詞接続,*,*,*,*,ぶっ,ブッ,ブッ
壊す
動詞,自立,*,*,五段・サ行,基本形,壊す,コワス,コワス

BOS/EOS,*,*,*,*,*,*,*,*


In [21]:
# Extract only nouns
neologd = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
text = '国会議員Youtuberの立花孝志がNHKとKaggleをぶっ壊す'
node = neologd.parseToNode(text)

while node:
    word=node.surface 
    feature =node.feature.split(',') # Use split to put each element in a list
    if feature[0] == "名詞":
        print(word)
        print(','.join(feature)) # Use join to put together every element of the list
    node = node.next

国会議員
名詞,固有名詞,一般,*,*,*,国会議員,コッカイギイン,コッカイギーン
Youtuber
名詞,固有名詞,一般,*,*,*,Youtuber,ユーチューバー,ユーチューバー
立花孝志
名詞,固有名詞,人名,一般,*,*,立花孝志,タチバナタカシ,タチバナタカシ
NHK
名詞,固有名詞,一般,*,*,*,NHK,エヌエイチケイ,エヌエイチケイ
Kaggle
名詞,一般,*,*,*,*,*


## Large text example

In [22]:
# Download Kokoro!
# !brew install wget
!wget -O kokoro.zip  https://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip
!unzip -f kokoro.zip

--2019-09-04 12:02:08--  https://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip
www.aozora.gr.jp (www.aozora.gr.jp) をDNSに問いあわせています... 59.106.13.115
www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 153688 (150K) [application/zip]
`kokoro.zip' に保存中

kokoro.zip          100%[===================>] 150.09K  --.-KB/s 時間 0.02s      

2019-09-04 12:02:08 (7.58 MB/s) - `kokoro.zip' へ保存完了 [153688/153688]

Archive:  kokoro.zip
Made with MacWinZipper™


In [21]:
import re
title='kokoro'
bindata = open(title+'.txt', 'rb').read()
textdata = bindata.decode('shift_jis')

# 青空文庫のための固有処理
textdata = re.split(r'\-{5,}', textdata)[2]
textdata = re.split(r'底本：', textdata)[0]
textdata = textdata.strip()

# 人によっては以下のパスは異なるので確認してね
mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
mecab.parse('')  # バグ対処
results = []
lines = textdata.split("\r\n")
for line in lines:
    r = []
    # 学習に使わない表現の削除処理
    s = line
    s = s.replace("|", "")
    s = re.sub(r'《.+?》', "", s)
    s = re.sub(r'［.+?］', '', s)
    # Mecab
    node = mecab.parseToNode(s)
    while node:
        # 単語を取得
        if node.feature.split(",")[6] == '*':
            word = node.surface
        else:
            word = node.feature.split(",")[6]

        # 品詞を取得
        part = node.feature.split(",")[0]

        if part in ["名詞", "形容詞", "動詞", "記号"]:
            r.append(word)
        node = node.next
    rl = (" ".join(r)).strip()
    results.append(rl)

# write to a file
w_file = title+"_result.txt"
with open(w_file, 'w', encoding='utf-8') as wf:
    wf.write("\n".join(results))

In [30]:
import pandas as pd
kokoro_pd = pd.read_csv(title+'.txt' ,header=None, encoding="SHIFT-JIS")
# Name a column as texts 
kokoro_pd.columns = ['texts']

In [31]:
# Find main texts
# Find the header ending row
start_index = kokoro_pd[kokoro_pd['texts'].str.contains(r'\-{5,}', regex=True)].index[1]+1
# Find the footer beginning row
end_index = kokoro_pd[kokoro_pd['texts'].str.contains(r'底本：', regex=True)].index[0]
kokoro_main_pd=kokoro_pd.iloc[start_index:end_index]
kokoro_main_pd.head()

,texts
12,［＃２字下げ］上 先生と私［＃「上 先生と私」は大見出し］
13,［＃５字下げ］一［＃「一」は中見出し］
14,私《わたくし》はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち...
15,私が先生と知り合いになったのは鎌倉《かまくら》である。その時私はまだ若々しい書生であった。...
16,学校の授業が始まるにはまだ大分《だいぶ》日数《ひかず》があるので鎌倉におってもよし、帰って...


In [32]:
# Clean out all rubi and comments

# Drop all rubi
kokoro_main_pd.loc[:,'texts']=kokoro_main_pd.loc[:,'texts'].str.replace(r'《.+?》', "", regex=True)
# Drop all comments
kokoro_main_pd.loc[:,'texts']=kokoro_main_pd.loc[:,'texts'].str.replace(r'［.+?］', "", regex=True)
# Drop zenkaku
kokoro_main_pd.loc[:,'texts']=kokoro_main_pd.loc[:,'texts'].str.replace(r'\u3000', " ", regex=True)
# Strip space
kokoro_main_pd.loc[:,'texts']=kokoro_main_pd.loc[:,'texts'].str.strip()
# Verify that there is no empty string
# kokoro_main_pd.loc[:,'texts'].str.find("").any()
# Reset index
kokoro_main_pd.reset_index(drop=True, inplace=True)

kokoro_main_pd.head(20)

/Users/yonetani/anaconda3/envs/analysis/lib/python3.7/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,texts
0,上 先生と私
1,一
2,私はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち明けない。これ...
3,私が先生と知り合いになったのは鎌倉である。その時私はまだ若々しい書生であった。暑中休暇を利用...
4,学校の授業が始まるにはまだ大分日数があるので鎌倉におってもよし、帰ってもよいという境遇にいた...
5,宿は鎌倉でも辺鄙な方角にあった。玉突きだのアイスクリームだのというハイカラなものには長い畷を...
6,私は毎日海へはいりに出掛けた。古い燻ぶり返った藁葺の間を通り抜けて磯へ下りると、この辺にこれ...
7,私は実に先生をこの雑沓の間に見付け出したのである。その時海岸には掛茶屋が二軒あった。私はふと...
8,二
9,私がその掛茶屋で先生を見た時は、先生がちょうど着物を脱いでこれから海へ入ろうとするところであ...


In [33]:
mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
mecab.parse('') # To avoid a bug

kokoro_main_pd.loc[:,'mecab']=kokoro_main_pd.loc[:,'texts'].apply(lambda x: mecab.parse(x).split())
kokoro_main_pd.loc[:,'wakati']=kokoro_main_pd.loc[:,'texts'].apply(lambda x: mecab.parse(x).split()[:-1:2])
kokoro_main_pd.loc[:,'syntax']=kokoro_main_pd.loc[:,'texts'].apply(lambda x: mecab.parse(x).split()[1::2])
kokoro_main_pd.head(10)

/Users/yonetani/anaconda3/envs/analysis/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/yonetani/anaconda3/envs/analysis/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,texts,mecab,wakati,syntax
0,上 先生と私,"[上, 名詞,一般,*,*,*,*,上,ウエ,ウエ, 先生, 名詞,一般,*,*,*,*,先...","[上, 先生, と, 私]","[名詞,一般,*,*,*,*,上,ウエ,ウエ, 名詞,一般,*,*,*,*,先生,センセイ,..."
1,一,"[一, 名詞,数,*,*,*,*,一,イチ,イチ, EOS]",[一],"[名詞,数,*,*,*,*,一,イチ,イチ]"
2,私はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち明けない。これ...,"[私, 名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, は, 助詞,係助詞,*,*,*...","[私, は, その, 人, を, 常に, 先生, と, 呼ん, で, い, た, 。, だか...","[名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, 助詞,係助詞,*,*,*,*,は,ハ..."
3,私が先生と知り合いになったのは鎌倉である。その時私はまだ若々しい書生であった。暑中休暇を利用...,"[私, 名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, が, 助詞,格助詞,一般,*,...","[私, が, 先生, と, 知り合い, に, なっ, た, の, は, 鎌倉, で, ある,...","[名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, 助詞,格助詞,一般,*,*,*,が,..."
4,学校の授業が始まるにはまだ大分日数があるので鎌倉におってもよし、帰ってもよいという境遇にいた...,"[学校, 名詞,一般,*,*,*,*,学校,ガッコウ,ガッコー, の, 助詞,連体化,*,*...","[学校, の, 授業, が, 始まる, に, は, まだ, 大分, 日数, が, ある, の...","[名詞,一般,*,*,*,*,学校,ガッコウ,ガッコー, 助詞,連体化,*,*,*,*,の,..."
5,宿は鎌倉でも辺鄙な方角にあった。玉突きだのアイスクリームだのというハイカラなものには長い畷を...,"[宿, 名詞,一般,*,*,*,*,宿,ヤド,ヤド, は, 助詞,係助詞,*,*,*,*,は...","[宿, は, 鎌倉, で, も, 辺鄙, な, 方角, に, あっ, た, 。, 玉突き, ...","[名詞,一般,*,*,*,*,宿,ヤド,ヤド, 助詞,係助詞,*,*,*,*,は,ハ,ワ, ..."
6,私は毎日海へはいりに出掛けた。古い燻ぶり返った藁葺の間を通り抜けて磯へ下りると、この辺にこれ...,"[私, 名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, は, 助詞,係助詞,*,*,*...","[私, は, 毎日, 海へ, は, いりに, 出掛け, た, 。, 古い, 燻, ぶり, 返...","[名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, 助詞,係助詞,*,*,*,*,は,ハ..."
7,私は実に先生をこの雑沓の間に見付け出したのである。その時海岸には掛茶屋が二軒あった。私はふと...,"[私, 名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, は, 助詞,係助詞,*,*,*...","[私, は, 実に, 先生, を, この, 雑沓, の, 間, に, 見付け, 出し, た,...","[名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, 助詞,係助詞,*,*,*,*,は,ハ..."
8,二,"[二, 名詞,数,*,*,*,*,二,ニ,ニ, EOS]",[二],"[名詞,数,*,*,*,*,二,ニ,ニ]"
9,私がその掛茶屋で先生を見た時は、先生がちょうど着物を脱いでこれから海へ入ろうとするところであ...,"[私, 名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, が, 助詞,格助詞,一般,*,...","[私, が, その, 掛茶屋, で, 先生, を, 見, た, 時, は, 、, 先生, が...","[名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ, 助詞,格助詞,一般,*,*,*,が,..."


In [36]:
# Use CountVectorizer
# Doc: https://qiita.com/chamao/items/7edaba62b120a660657e
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')

feature_vectors = count_vectorizer.fit_transform(kokoro_main_pd.loc[:,'wakati'].apply(lambda x: ' '.join(x)))
vocabulary = count_vectorizer.get_feature_names()
vocabulary[0:10]


['々', 'あ', 'ああ', 'ああなっ', 'あえて', 'あか', 'あからさま', 'あきれ', 'あくび', 'あくまで']

## Add words into dictionary

In [37]:
%%bash
# Look up dictionary folder 
cd $(mecab-config --dicdir)
pwd

# Make a user dictionary in a CSV format
cd $(mecab-config --dicdir )
mkdir -p userdic
cd userdic
cat > add.csv << EOF
ITトレンド,,,20,名詞,一般,*,*,*,*,ITトレンド,アイティートレンド,アイティートレンド
EOF
#Check the result
cat add.csv

# Complile the file to the dictionary
$(mecab-config --libexecdir)/mecab-dict-index \
-d /usr/local/lib/mecab/dic/ipadic \
-u /usr/local/lib/mecab/dic/userdic/add.dic \
-f utf-8 \
-t utf-8 \
add.csv

/usr/local/lib/mecab/dic
ITトレンド,,,20,名詞,一般,*,*,*,*,ITトレンド,アイティートレンド,アイティートレンド
reading add.csv ... 1
emitting double-array: 100% |###########################################| 

done!


In [ ]:
%%bash
original_userdic_path='\;\s*userdic.*$'
new_userdic_path='userdic=\/usr\/local\/lib\/mecab\/dic\/userdic\/add.dic'
sed -i -e "s/$original_userdic_path/$new_userdic_path/g" $(mecab-config --sysconfdir)'/mecabrc'
# Multiple dictionaries can be specified like the below
# userdic = /home/foo/bar/human.dic,/home/foo/bar/manga.dic

# Show mecabrc
cat $(mecab-config --sysconfdir)'/mecabrc'

In [38]:
# Command line
!echo "ITトレンド" | mecab

ITトレンド	名詞,一般,*,*,*,*,ITトレンド,アイティートレンド,アイティートレンド
EOS


In [39]:
# Python

# Default
# neologd = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -O wakati' )
# neologd.parse("2020年のITトレンド").split()

neologd = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -u /usr/local/lib/mecab/dic/userdic/add.dic -O wakati')
neologd.parse("2020年のITトレンド").split()

['2020年', 'の', 'ITトレンド']

# Wordnet in Japanese
Official Doc: http://compling.hss.ntu.edu.sg/wnja/

In [32]:
%%bash
# Japanese Wordnet and English WordNet in an sqlite3 database
mkdir -p wordnet&&cd $_
wget http://compling.hss.ntu.edu.sg/wnja/data/1.1/wnjpn.db.gz
gunzip -f wnjpn.db.gz

--2019-09-04 12:02:10--  http://compling.hss.ntu.edu.sg/wnja/data/1.1/wnjpn.db.gz
compling.hss.ntu.edu.sg (compling.hss.ntu.edu.sg) をDNSに問いあわせています... 155.69.254.223
compling.hss.ntu.edu.sg (compling.hss.ntu.edu.sg)|155.69.254.223|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 60390049 (58M) [application/x-gzip]
`wnjpn.db.gz' に保存中

     0K .......... .......... .......... .......... ..........  0%  313K 3m8s
    50K .......... .......... .......... .......... ..........  0%  507K 2m32s
   100K .......... .......... .......... .......... ..........  0%  615K 2m13s
   150K .......... .......... .......... .......... ..........  0%  594K 2m4s
   200K .......... .......... .......... .......... ..........  0%  614K 1m59s
   250K .......... .......... .......... .......... ..........  0%  624K 1m54s
   300K .......... .......... .......... .......... ..........  0%  629K 1m51s
   350K .......... .......... .......... .......... ..........  0%  628K 1m49s
   400K ........

In [41]:
import sqlite3
conn = sqlite3.connect("wordnet/wnjpn.db")

In [42]:
# Check tables
tables_df=pd.read_sql_query("select name from sqlite_master where type='table' ", conn)
tables_df

,name
0,pos_def
1,link_def
2,synset_def
3,synset_ex
4,synset
5,synlink
6,ancestor
7,sense
8,word
9,variant


## Word Table

In [43]:
# Check columns in word
    
word_table_df=pd.read_sql_query("PRAGMA TABLE_INFO(word)", conn)
word_table_df

,cid,name,type,notnull,dflt_value,pk
0,0,wordid,integer,0,None,1
1,1,lang,text,0,None,0
2,2,lemma,text,0,None,0
3,3,pron,text,0,None,0
4,4,pos,text,0,None,0


In [45]:
# Words in Japanese
words_df=pd.read_sql_query("select * from word where lang='jpn'", conn)
words_df.head()

,wordid,lang,lemma,pron,pos
0,155288,jpn,頭金,None,n
1,155289,jpn,どうにかこうにか,None,r
2,155290,jpn,大砲,None,n
3,155291,jpn,スチーム,None,n
4,155292,jpn,溢れでる,None,v


In [57]:
# How many words in the DB?
word_counts_df=pd.read_sql_query("select count(*) from word", conn)
word_counts_df

,count(*)
0,249121


## Sense Table

In [58]:
# Check columns in sense
sense_table_df=pd.read_sql_query("PRAGMA TABLE_INFO(sense)", conn)
sense_table_df

,cid,name,type,notnull,dflt_value,pk
0,0,synset,text,0,None,0
1,1,wordid,integer,0,None,0
2,2,lang,text,0,None,0
3,3,rank,text,0,None,0
4,4,lexid,integer,0,None,0
5,5,freq,integer,0,None,0
6,6,src,text,0,None,0


In [63]:
# Sense in Japanese
sense_df=pd.read_sql_query("select * from sense where lang='jpn' order by synset", conn)
sense_df.head(10)

,synset,wordid,lang,rank,lexid,freq,src
0,00001740-a,226093,jpn,None,None,None,hand
1,00001740-n,209968,jpn,None,None,None,hand
2,00001740-r,201261,jpn,None,None,None,hand
3,00001740-v,164161,jpn,None,None,None,hand
4,00001740-v,176643,jpn,None,None,None,hand
5,00001740-v,178937,jpn,None,None,None,hand
6,00001740-v,186954,jpn,None,None,None,hand
7,00001740-v,216393,jpn,None,None,None,hand
8,00001837-r,221066,jpn,None,None,None,mono
9,00002098-a,162575,jpn,None,None,None,hand


In [49]:
# Extract words which have the same concept
sense_synset1_df=pd.read_sql_query("select * from word where wordid in (select wordid from sense where synset = '00001740-n') and lang='jpn' ", conn)
sense_synset1_df.head(10)


# Another example    
# sense_synset2_df=pd.read_sql_query("select * from word where wordid in (select wordid from sense where synset='00002621-r') and lang='jpn' ", conn)
# sense_synset2_df.head(10)

,wordid,lang,lemma,pron,pos
0,209968,jpn,実体,None,n


## Synset Table

In [69]:
# Check columns in synset
synset_table_df=pd.read_sql_query("PRAGMA TABLE_INFO(synset) ", conn)
synset_table_df.head(10)


,cid,name,type,notnull,dflt_value,pk
0,0,synset,text,0,None,0
1,1,pos,text,0,None,0
2,2,name,text,0,None,0
3,3,src,text,0,None,0


In [70]:
# Synset examples    
synset_table_df=pd.read_sql_query("select * from synset order by synset", conn)
synset_table_df.head(10)

,synset,pos,name,src
0,00001740-a,a,able,eng30
1,00001740-n,n,entity,eng30
2,00001740-r,r,a_cappella,eng30
3,00001740-v,v,breathe,eng30
4,00001837-r,r,anno_domini,eng30
5,00001930-n,n,physical_entity,eng30
6,00001981-r,r,c.e.,eng30
7,00002098-a,a,unable,eng30
8,00002137-n,n,abstract_entity,eng30
9,00002142-r,r,bc,eng30


In [50]:
# Extract the concept name of '00001740-v'

synset_example_df=pd.read_sql_query("select * from synset WHERE synset='00001740-v' ", conn)
synset_example_df.head(10)

,synset,pos,name,src
0,00001740-v,v,breathe,eng30


## Synset Definition

In [51]:
# Synset Def examples    
synset_def_df=pd.read_sql_query("select * from synset_def WHERE lang='jpn'", conn)
synset_def_df.head(10)

,synset,lang,def,sid
0,01785341-a,jpn,懇願によって静められない、なだめられない、または、動かされないさま,0
1,11820323-n,jpn,南アフリカ産の茎のない多肉植物の属,0
2,04239436-n,jpn,傷ついた前腕を支持する包帯,0
3,04239436-n,jpn,首のあたりから掛かっている布の広い三角形の部分からなるもの,1
4,11427067-n,jpn,太陽の大気圏の最も外側の領域,0
5,11427067-n,jpn,日食の間、白色の暈として見ることができる,1
6,14069747-n,jpn,サルモネラ菌に汚染された食物を摂取することで生じる食中毒,0
7,01614195-n,jpn,タカ科の属,0
8,00102786-a,jpn,2つの通常相容れない考え方で解釈できるさま,0
9,00307333-r,jpn,不快で吐き気がするさま,0


# Synonym Finder

In [74]:
# https://qiita.com/pocket_kyoto/items/1e5d464b693a8b44eda5
# 特定の単語を入力とした時に、類義語を検索する関数
# Better to use try/except syntax? https://www.dreamincode.net/forums/topic/87732-tryexcept-vs-ifelse/
def SearchSimilarWords(word):

    # 問い合わせしたい単語がWordnetに存在するか確認する
    cur = conn.execute("select wordid from word where lemma='%s'" % word)
    word_id = 99999999  #temp 
    for row in cur:
        word_id = row[0]

    # Wordnetに存在する語であるかの判定
    if word_id==99999999:
        print("「%s」は、Wordnetに存在しない単語です。" % word)
        return
    else:
        print("【「%s」の類似語を出力します】\n" % word)

    # 入力された単語を含む概念を検索する
    cur = conn.execute("select synset from sense where wordid='%s'" % word_id)
    synsets = []
    for row in cur:
        synsets.append(row[0])

    # 概念に含まれる単語を検索して画面出力する
    no = 1
    for synset in synsets:
        cur1 = conn.execute("select name from synset where synset='%s'" % synset)
        for row1 in cur1:
            print("%sつめの概念 : %s" %(no, row1[0]))
        cur2 = conn.execute("select def from synset_def where (synset='%s' and lang='jpn')" % synset)
        sub_no = 1
        for row2 in cur2:
            print("意味%s : %s" %(sub_no, row2[0]))
            sub_no += 1
        cur3 = conn.execute("select wordid from sense where (synset='%s' and wordid!=%s)" % (synset,word_id))
        sub_no = 1
        for row3 in cur3:
            target_word_id = row3[0]
            cur3_1 = conn.execute("select lemma from word where wordid=%s" % target_word_id)
            for row3_1 in cur3_1:
                print("類義語%s : %s" % (sub_no, row3_1[0]))
                sub_no += 1
        print("\n")
        no += 1

In [79]:
# Example
SearchSimilarWords("ネコ")

【「ネコ」の類似語を出力します】

1つめの概念 : true_cat
意味1 : 通常、厚く柔らかい毛皮を持ち、吠えることのできないネコ科の哺乳類：家ネコ
意味2 : ヤマネコ
類義語1 : true_cat
類義語2 : cat
類義語3 : ねんねこ
類義語4 : にゃんにゃん
類義語5 : 猫
類義語6 : キャット




In [100]:
# Synset links (Taxonomy)
# Ref: https://www.cs.cmu.edu/~hideki/software/jawjaw/index-en.html
synlink_df=pd.read_sql_query("select * from synlink", conn)
synlink_df.head(10)


,synset1,synset2,link,src
0,07125096-n,07128527-n,hype,eng30
1,07126228-n,07109847-n,hype,eng30
2,14123044-n,14122235-n,hype,eng30
3,14123044-n,14123259-n,hypo,eng30
4,08030185-n,08197895-n,inst,eng30
5,09902017-n,10658501-n,hype,eng30
6,03259505-n,03546340-n,hype,eng30
7,03259505-n,03042490-n,hypo,eng30
8,03259505-n,03088580-n,hypo,eng30
9,03259505-n,03354207-n,hypo,eng30


In [110]:
# 07125096-n (hyponymy)and 07128527-n (hypernymy)
relation_example1_df=pd.read_sql_query("select * from word where wordid in (select wordid from sense where synset = '07125096-n')", conn)
relation_example1_df.head(20)


,wordid,lang,lemma,pron,pos
0,1,eng,expletive,None,n
1,14163,eng,swearing,None,n
2,29488,eng,oath,None,n
3,89063,eng,cuss,None,n
4,101879,eng,swearword,None,n
5,102167,eng,curse_word,None,n
6,106475,eng,curse,None,n
7,155512,jpn,悪罵,None,n
8,159965,jpn,罰当たりな言葉,None,n
9,175679,jpn,罵り,None,n


In [111]:
relation_example2_df=pd.read_sql_query("select * from word where wordid in (select wordid from sense where synset = '07128527-n')", conn)
relation_example2_df.head(10)

,wordid,lang,lemma,pron,pos
0,64742,eng,profanity,None,n
1,156441,jpn,不敬,None,n
2,158636,jpn,冒とく,None,n
3,201882,jpn,冒涜,None,n


4. countvectorizer
5. word2vec